In [3676]:
#!pip install lightgbm
#import tensorflow
print(tensorflow.__version__)

2.9.1


In [3677]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn import preprocessing

%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import zero_one_loss
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

 
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
import xgboost as XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
import xgboost as XGB
from xgboost import XGBClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier

In [3678]:
df=pd.read_csv(r"C:\\Users\Abduallah Hussien\\Desktop\Untitled Folder 1\\milestone 2\\train.csv")
df_test=pd.read_csv(r'C:\\Users\Abduallah Hussien\\Desktop\Untitled Folder 1\\milestone 2\\test.csv')
allData = [df, df_test]

In [3679]:
df_all=pd.concat([df,df_test])
df_all

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,10497,Male,Yes,33,No,Engineer,0.0,Average,2.0,Cat_6,B
1,5748,Female,No,42,Yes,Doctor,5.0,Low,1.0,Cat_6,B
2,4228,Male,Yes,80,No,Executive,0.0,High,2.0,Cat_6,A
3,10369,Female,Yes,66,Yes,Artist,0.0,Average,3.0,Cat_6,C
4,10442,Female,Yes,39,Yes,Healthcare,8.0,High,2.0,Cat_1,D
...,...,...,...,...,...,...,...,...,...,...,...
3525,4323,Male,Yes,79,Yes,Artist,NaN,High,2.0,Cat_6,NaN
3526,3776,Male,Yes,40,Yes,Entertainment,1.0,Average,2.0,Cat_3,NaN
3527,6697,Male,Yes,73,Yes,Artist,5.0,High,3.0,Cat_6,NaN
3528,3431,Male,No,20,No,Healthcare,NaN,Low,5.0,Cat_3,NaN


In [3680]:
for i in allData:
    i['Family_Size']=i['Family_Size'].fillna(df_all['Family_Size'].mean())
    i['Ever_Married']=i['Ever_Married'].fillna(df_all['Ever_Married'].mode()[0])
    i['Profession']=i['Profession'].fillna(df_all['Profession'].mode()[0])
    i['Graduated']=i['Graduated'].fillna(df_all['Graduated'].mode()[0])
    i['Var_1']=i['Var_1'].fillna(df_all['Var_1'].mode()[0])
    i['Work_Experience']=i['Work_Experience'].fillna(15)
    df.loc[(df['Work_Experience'] ==15) & (df['Ever_Married'] =="No")&(df['Segmentation'] =="D"), 'Work_Experience'] =0
    df.loc[(df['Work_Experience'] ==15) & (df['Age'] <30)&(df['Segmentation'] =="D"), 'Work_Experience'] =0
    df.loc[(df['Work_Experience'] ==15) & (df['Age'] >63)&(df['Segmentation'] =="D") & (df['Family_Size'] ==1) , 'Work_Experience'] =0
    i.loc[(i['Ever_Married'] =="No") &(i['Graduated'] =="No") & (i['Work_Experience'] ==15) , 'Work_Experience'] =0
    i.loc[(i['Profession'] =="Homemaker") &(i['Work_Experience'] ==15) , 'Work_Experience'] =9
    i.loc[(i['Profession'] =="Lawyer") &(i['Age'] >65) &(i['Graduated'] =="No") &(i['Work_Experience'] ==15), 'Work_Experience'] =0
    i.loc[(i['Work_Experience'] ==15) & (i['Profession'] =="Marketing"), 'Work_Experience'] =0
    i.loc[(i['Work_Experience'] ==15) & (i['Profession'] =="Doctor")&(i['Age'] >25) & (i['Age'] <40), 'Work_Experience'] =0
    i.loc[(i['Gender'] =="Female") &(i['Ever_Married'] =="No")&(i['Age'] >28) &(i['Work_Experience'] ==15), 'Work_Experience'] =0
    i.loc[(i['Work_Experience'] ==15), 'Work_Experience'] =1
    i['Graduated']=i['Graduated'].map( {"Yes": 1, "No": 2} ).astype(int)
    i['Ever_Married']=i['Ever_Married'].map( {"Yes": 1, "No": 0} ).astype(int)
    i['Spending_Score']=i['Spending_Score'].map( {"High": 1, "Average": 2,"Low":3} ).astype(int)
    i['Gender']=i['Gender'].map( {"Male": 1, "Female": 2} ).astype(int)
    i['Var_1']=i['Var_1'].map( {"Cat_6": 6, "Cat_5": 5,"Cat_4":4,"Cat_3":3,"Cat_2":2,"Cat_1":1,"Cat_7": 7} ).astype(int)
    i['Profession']=i['Profession'].map( {"Artist": 1, "Healthcare": 2,"Entertainment":3,"Doctor":4,"Engineer":5,"Lawyer":6,"Executive": 7,"Marketing":8,"Homemaker":9} ).astype(int)
df['Segmentation']=df['Segmentation'].map( {"A": 0, "B": 1,"C":2,"D":3} ).astype(int)


In [3681]:
df=pd.get_dummies(df,prefix='Gender',columns=['Gender'],drop_first=True)
df['Ever_Married']=df['Ever_Married'].fillna('Yes')
df=pd.get_dummies(df,prefix='Married',columns=['Ever_Married'],drop_first=True)
df['Graduated']=df['Graduated'].fillna('Yes')
df=pd.get_dummies(df,prefix='Graduated',columns=['Graduated'],drop_first=True)
df['Profession'].fillna('missing',inplace=True)
label_encoder = preprocessing.LabelEncoder()
df['Profession']=label_encoder.fit_transform(df['Profession'])
df.drop('Profession',axis=1,inplace=True)
df['Work_Experience'].fillna(df['Work_Experience'].mean(),inplace=True)
df['Spending_Score']=df['Spending_Score'].map( {"High": 3, "Average": 2,"Low":1} ).astype('int')
df['Family_Size'].fillna(round(df['Family_Size'].mean()),inplace=True)
df['Var_1'].fillna('Cat_6',inplace=True)
df['Var_1']=label_encoder.fit_transform(df['Var_1'])
df['Var_1']=df['Var_1'].astype('int')
df['Spending_Score']=df['Spending_Score'].astype('int')
df.head()

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
df_test=pd.get_dummies(df_test,prefix='Gender',columns=['Gender'],drop_first=True)
df_test['Ever_Married']=df_test['Ever_Married'].fillna('Yes')
df_test=pd.get_dummies(df_test,prefix='Married',columns=['Ever_Married'],drop_first=True)
df_test['Graduated']=df_test['Graduated'].fillna('Yes')
df_test=pd.get_dummies(df_test,prefix='Graduated',columns=['Graduated'],drop_first=True)
df_test['Profession'].fillna('missing',inplace=True)
label_encoder = preprocessing.LabelEncoder()
df_test['Profession']=label_encoder.fit_transform(df_test['Profession'])
df_test.drop('Profession',axis=1,inplace=True)
df_test['Work_Experience'].fillna(df_test['Work_Experience'].mean(),inplace=True)
df_test['Spending_Score']=df_test['Spending_Score'].map( {"High": 3, "Average": 2,"Low":1} ).astype('int')
df_test['Family_Size'].fillna(round(df_test['Family_Size'].mean()),inplace=True)
df_test['Var_1'].fillna('Cat_6',inplace=True)
df_test['Var_1']=label_encoder.fit_transform(df_test['Var_1'])
df_test['Var_1']=df_test['Var_1'].astype('int')
df_test['Spending_Score']=df_test['Spending_Score'].astype('int')
df_test

In [ ]:
df.drop([149, 764, 2729, 3301, 4528, 5980,1810,3560,6072],inplace=True)

In [ ]:
df['Age']=np.sqrt(df['Age'])
df['Work_Experience']=np.sqrt(df['Work_Experience'])
df['Family_Size']=np.sqrt(df['Family_Size'])
df_test['Age']=np.sqrt(df_test['Age'])
df_test['Work_Experience']=np.sqrt(df_test['Work_Experience'])
df_test['Family_Size']=np.sqrt(df_test['Family_Size'])

In [ ]:
label_encoder = preprocessing.LabelEncoder()
df['Segmentation']=label_encoder.fit_transform(df['Segmentation'])
df

In [ ]:
Inter_Quartile_Range = df['Family_Size'].quantile(0.75) - df['Family_Size'].quantile(0.25)

lower_boundary = df['Family_Size'].quantile(0.25) - (Inter_Quartile_Range * 1.5)
upper_boundary = df['Family_Size'].quantile(0.75) + (Inter_Quartile_Range * 1.5)
df['Family_Size']= np.where(df['Family_Size'] > upper_boundary, upper_boundary,np.where(df['Family_Size'] < lower_boundary, lower_boundary,df['Family_Size']))

In [ ]:
X=df.drop(['Segmentation','ID'],axis=1)
y=df['Segmentation']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=1)


In [ ]:
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# n_estimators = [100, 300, 500, 800, 1200]
# max_depth = [5, 8, 15, 25, 30]
# min_samples_split = [2, 5, 10, 15, 100]
# min_samples_leaf = [1, 2, 5, 10] 
# forest= RandomForestClassifier()

# hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
#               min_samples_split = min_samples_split, 
#              min_samples_leaf = min_samples_leaf)

# gridF = RandomizedSearchCV(forest, hyperF, cv = 3, verbose = 1, 
#                       n_jobs = -1)
# bestF = gridF.fit(X_train, y_train)

In [ ]:
print("Best depth:",bestF.best_estimator_.get_params()['max_depth'])
print("Best n_estimators:",bestF.best_estimator_.get_params()['n_estimators'])
print("Best min_samples_split:",bestF.best_estimator_.get_params()['min_samples_split'])
print("Best min_samples_leaf:",bestF.best_estimator_.get_params()['min_samples_leaf'])

In [ ]:
from sklearn.metrics import mean_squared_error
lgb = lgb.LGBMClassifier(subsample=0.4, reg_lambda=0.2, reg_alpha=1.4,
               random_state=1, objective='multiclass', max_depth=18,
               min_data_in_leaf=55, learning_rate=0.04, colsample_bytree=0.8,
               n_estimators=4000, boosting_type='gbdt'
)

lgb.fit(X_train, y_train,early_stopping_rounds=100,eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_metric='multi_error', verbose=True)


# predicting the output on the validation dataset
Y_pred_lgb = lgb.predict(X_test)
Y_pred2 = lgb.predict(X_train)
Y_pred3 = lgb.predict(X_val)

# printing the root mean squared error between real value and predicted value
print('Accuracy of model on train set: {:.2f}'.format(lgb.score(X_train, y_train)))
print('Accuracy of model on test set: {:.2f}'.format(lgb.score(X_val, y_val)))
print('mse of model on train set: {:.2f}'.format(np.sqrt(mean_squared_error(y_train, Y_pred2))))
print('mse of model on test set: {:.2f}'.format(np.sqrt(mean_squared_error(y_val, Y_pred3))))

In [ ]:
X_test=df_test.drop(['ID'],axis=1)
iD=df_test['ID']

In [ ]:
from sklearn.metrics import mean_squared_error
rf = RandomForestClassifier(n_estimators=1600, min_samples_split=2, min_samples_leaf=4, max_features='sqrt', max_depth=5, bootstrap=True) 

rf.fit(X_train, y_train)


# predicting the output on the validation dataset
Y_pred_rf = rf.predict(X_test)
Y_pred2 = rf.predict(X_train)
Y_pred3 = rf.predict(X_val)

# printing the root mean squared error between real value and predicted value
print('Accuracy of model on train set: {:.2f}'.format(rf.score(X_train, y_train)))
print('Accuracy of model on test set: {:.2f}'.format(rf.score(X_val, y_val)))
print('mse of model on train set: {:.2f}'.format(np.sqrt(mean_squared_error(y_train, Y_pred2))))
print('mse of model on test set: {:.2f}'.format(np.sqrt(mean_squared_error(y_val, Y_pred3))))

In [ ]:
# parameters = {'iterations':np.arange(1000,5000,1000), 'learning_rate':np.arange(0,0.1,0.01), 'objective':['Logloss','MultiClass','MultiClassOneVsAll'],
#              'max_depth':np.arange(1,10,1),'random_strength':[0,1,2,3,4,5],'random_state':[42,0,1]}
# model = CatBoostClassifier()
# clf = RandomizedSearchCV(estimator=model, param_distributions=parameters,cv=5,verbose=3)
# bm = clf.fit(X_train, y_train)
# #sorted(clf.cv_results_)
# print(bm.best_params_)

In [ ]:
# initializing all the model objects with default parameters
cat = CatBoostClassifier(random_strength=3, random_state=1, objective='MultiClass',max_depth=2, iterations=4000,learning_rate=0.06, silent=True)
# training all the model on the training dataset
cat.fit(X_train, y_train)
# predicting the output on the validation dataset
Y_pred = cat.predict(X_test)
Y_pred2 = cat.predict(X_train)
Y_pred3 = cat.predict(X_val)

# printing the root mean squared error between real value and predicted value
print('Accuracy of model on train set: {:.2f}'.format(cat.score(X_train, y_train)))
print('Accuracy of model on test set: {:.2f}'.format(cat.score(X_val, y_val)))
print('mse of model on train set: {:.2f}'.format(np.sqrt(mean_squared_error(y_train, Y_pred2))))
print('mse of model on test set: {:.2f}'.format(np.sqrt(mean_squared_error(y_val, Y_pred3))))

In [ ]:
# params={"n_estimators":[67,70,100,120],'reg_lambda':[2,1],'gamma':[0,0.3,0.2,0.1]
#        ,'eta':[0.06,0.05,0.04]
#         ,"max_depth":[3,5],'objective':['binary:logistic']}
# reg=XGBClassifier()
# clf=RandomizedSearchCV(reg,params,cv=10,n_jobs=-1,verbose=1)
# bm=clf.fit(X_train,y_train)
# print(bm.best_params_)

In [ ]:
xgb = XGBClassifier(reg_lambda=1, objective='binary:logistic', n_estimators=100, max_depth=3, gamma=0,eta=0.04) 
# training all the model on the training dataset
xgb.fit(X_train, y_train)
# predicting the output on the validation dataset
Y_pred_xgb = xgb.predict(X_test)
Y_pred2 = xgb.predict(X_train)
Y_pred3 = xgb.predict(X_val)

# printing the root mean squared error between real value and predicted value
print('Accuracy of model on train set: {:.2f}'.format(xgb.score(X_train, y_train)))
print('Accuracy of model on test set: {:.2f}'.format(xgb.score(X_val, y_val)))
print('mse of model on train set: {:.2f}'.format(np.sqrt(mean_squared_error(y_train, Y_pred2))))
print('mse of model on test set: {:.2f}'.format(np.sqrt(mean_squared_error(y_val, Y_pred3))))

In [ ]:
ada=AdaBoostClassifier(n_estimators=3,learning_rate=1, random_state=42)
# training all the model on the training dataset
ada.fit(X_train, y_train)
# predicting the output on the validation dataset
Y_pred = ada.predict(X_test)
Y_pred2 = ada.predict(X_train)
Y_pred3 = ada.predict(X_val)

# printing the root mean squared error between real value and predicted value
print('Accuracy of model on train set: {:.2f}'.format(ada.score(X_train, y_train)))
print('Accuracy of model on test set: {:.2f}'.format(ada.score(X_val, y_val)))
print('mse of model on train set: {:.2f}'.format(np.sqrt(mean_squared_error(y_train, Y_pred2))))
print('mse of model on test set: {:.2f}'.format(np.sqrt(mean_squared_error(y_val, Y_pred3))))

In [ ]:
# parameters = {
#     "loss":["deviance"],
#     "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
#     "min_samples_split": np.linspace(0.1, 0.5, 12),
#     "min_samples_leaf": np.linspace(0.1, 0.5, 12),
#     "max_depth":[3,5,8],
#     "max_features":["log2","sqrt"],
#     "criterion": ["friedman_mse",  "mae"],
#     "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
#     "n_estimators":[10]
#     }

# clf = RandomizedSearchCV(GradientBoostingClassifier(), parameters, cv=10, n_jobs=-1)
# bm = clf.fit(X_train, y_train)
# #sorted(clf.cv_results_)
# print(bm.best_params_)

In [ ]:
gb=GradientBoostingClassifier(subsample=0.9,n_estimators=10,min_samples_split=0.17272727272727273,min_samples_leaf=0.2090909090909091,
                             max_features='log2',max_depth=3,loss='deviance',learning_rate=0.15,criterion='friedman_mse')
# training all the model on the training dataset
gb.fit(X_train, y_train)
# predicting the output on the validation dataset
Y_pred = gb.predict(X_test)
Y_pred2 = gb.predict(X_train)
Y_pred3 = gb.predict(X_val)

# printing the root mean squared error between real value and predicted value
print('Accuracy of model on train set: {:.2f}'.format(gb.score(X_train, y_train)))
print('Accuracy of model on test set: {:.2f}'.format(gb.score(X_val, y_val)))
print('mse of model on train set: {:.2f}'.format(np.sqrt(mean_squared_error(y_train, Y_pred2))))
print('mse of model on test set: {:.2f}'.format(np.sqrt(mean_squared_error(y_val, Y_pred3))))

In [ ]:
pred = pd.DataFrame(Y_pred_lgb)
data = pd.concat([iD,pred],axis = 1)
data.columns = ['ID','Segmentation']

In [ ]:
data['Segmentation'] = data['Segmentation'].map( {0: "A",  1: "B",2:"C",3:"D"} ).astype(object)

In [ ]:
data['Segmentation'].value_counts()

In [ ]:
data.to_csv (r"C:\Users\Abduallah Hussien\Desktop\Untitled Folder 1\milestone 2\submission_lgb.csv", index = False, header=True)
data

In [ ]:
df['Segmentation'].value_counts()

In [ ]:
#df = pd.read_csv("submission.csv",na_values=' ')
#df2 = pd.read_csv("0.47.csv",na_values=' ')
#x=df['Segmentation']
#x2=df2['Segmentation']
#x2

In [ ]:
#print('mse of two models gbr: {:.2f}'.format(np.sqrt(mean_squared_error(x2, x))))